In [72]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [2]:
df_org = pd.read_csv('docs/t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

C:\Users\anees\AppData\Local\Temp\ipykernel_23060\3596150139.py:1: DtypeWarning: Columns (54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_org = pd.read_csv('docs/t20s_2013-23_mgd.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])


In [3]:
df_org=df_org[(df_org['ball']<=11)]
df_org=df_org[(df_org['over']<=20)]
df_org=df_org.drop(columns='ball.1')
df_org=df_org[(df_org['batruns']>=0) & (df_org['bowlruns']>=0)]
df_org.shape

(1554670, 54)

In [4]:
def bowl_style_clean(df,unique_styles):

    df['bowl_style'] = df['bowl_style'].replace(['RFM', 'RMF'], 'RFM')
    df['bowl_style'] = df['bowl_style'].replace(['LFM', 'LMF'], 'LFM')
    df['bowl_style'] = df['bowl_style'].replace(['LBG', 'LB'], 'RWS')
    df['bowl_style'] = df['bowl_style'].replace(['LS', 'SLA'], 'SLA')
    df['bowl_style'] = df['bowl_style'].replace(['RAB'], 'RSM')
    df['bowl_style'] = df['bowl_style'].replace(['LAB'], 'LSM')
    unique_styles = ['LFM/SLA', 'OB/LB','LM/SLA/LWS','OB/SLA','RMF/OB', 'RFM/OB', 'OB/LBG', 'RFM/LBG', 'RM/LB',
           'RM/LBG', 'LMF/RM', 'RM/RSM', 'SLA/LWS', 'RMF/LB', 'RFM/LB',
           'RM/OB/LB']
    replacements = {
        'RFM': 'RFM',
        'RMF': 'RFM',
        'LFM': 'LFM',
        'LMF': 'LFM',
        'LBG': 'RWS',
        'LB': 'RWS',
        'LS': 'SLA',
        'SLA': 'SLA',
        'RAB': 'RS',
        'LAB': 'LS'
    }

    # Iterate over unique styles and apply replacements
    for style in unique_styles:
        replaced_style = '/'.join([replacements.get(part, part) for part in style.split('/')])
        df['bowl_style'] = df['bowl_style'].replace([style], replaced_style)
    #df['bowl_style'].unique()
    return df
unique_styles = ['RFM', 'RMF', 'LBG', 'LB', 'LS', 'RAB', 'LAB', 'OB', 'SLA', 'RAB','RSM','LSM']
df=bowl_style_clean(df_org,unique_styles)
df['bowl_style'].unique()

array(['RF', 'RFM', 'RWS', 'LWS', 'SLA', 'OB', 'LFM', 'LF', 'RM', 'RM/OB',
       'LM', 'LFM/SLA', 'OB/RWS', 'LM/SLA/LWS', 'OB/SLA', 'RSM', '-',
       'LSM', 'RFM/OB', 'RFM/RWS', 'RM/RWS', 'LFM/RM', 'RM/RSM',
       'SLA/LWS', 'RM/OB/RWS', 'RS'], dtype=object)

In [5]:
df_worldcup = pd.read_csv('docs/2024 wc squad.csv', encoding='latin1')
df_worldcup.head()

Afghanistan       Australia             Bangladesh           Canada  \
0         Rashid Khan  Mitchell Marsh  Najmul Hossain Shanto   Saad Bin Zafar   
1      Ibrahim Zadran     Ashton Agar           Taskin Ahmed    Aaron Johnson   
2  Azmatullah Omarzai     Pat Cummins             Litton Das  Dillon Heyliger   
3   Najibullah Zadran       Tim David          Soumya Sarkar   Dilpreet Bajwa   
4      Mohammad Ishaq    Nathan Ellis           Tanzid Hasan     Harsh Thaker   

         England              India         Ireland             Namibia  \
0     Jos Buttler      Rohit Sharma   Paul Stirling     Gerhard Erasmus   
1       Moeen Ali     Hardik Pandya      Mark Adair          Zane Green   
2    Jofra Archer  Yashasvi Jaiswal      Ross Adair  Michael van Lingen   
3  Jonny Bairstow       Virat Kohli  Andy Balbirnie       Dylan Leicher   
4     Harry Brook  Suryakumar Yadav  Curtis Campher   Ruben Trumpelmann   

            Nepal    Netherlands        New Zealand               Oman  \
0    Rohit Paudel  Scott Edwards    Kane Williamson         Aqib Ilyas   
1    Aasif Sheikh     Aryan Dutt         Finn Allen    Zeeshan Maqsood   
2  Anil Kumar Sah   Bas de Leede        Trent Boult  Kashyap Prajapati   
3  Kushal Bhurtel   Daniel Doram  Michael Bracewell    Pratik Athavale   
4    Kushal Malla  Fred Klaassen       Mark Chapman         Ayaan Khan   

  Papua New Guinea           Scotland      South Africa          Sri Lanka  \
0       Assad Vala  Richie Berrington     Aiden Markram  Wanindu Hasaranga   
1         Alei Nao      Matthew Cross  Ottniel Baartman   Charith Asalanka   
2       Chad Soper     Bradley Currie    Gerald Coetzee       Kusal Mendis   
3    Charles Amini      Chris Greaves   Quinton de Kock    Pathum Nissanka   
4        Hila Vare         Oli Carter     Bjorn Fortuin     Kamindu Mendis   

           Uganda   United States      West Indies  Pakistan  
0    Brian Masaba    Monank Patel    Rovman Powell       NaN  
1   Simon Ssesazi     Aaron Jones   Alzarri Joseph       NaN  
2    Roger Mukasa    Andries Gous  Johnson Charles       NaN  
3  Cosmas Kyewuta  Corey Anderson     Roston Chase       NaN  
4  Dinesh Nakrani        Ali Khan  Shimron Hetmyer       NaN

In [6]:
team_dict = df_worldcup.to_dict(orient='list')

print(team_dict['Afghanistan'])

['Rashid Khan', 'Ibrahim Zadran', 'Azmatullah Omarzai', 'Najibullah Zadran', 'Mohammad Ishaq', 'Mohammad Nabi', 'Gulbadin Naib', 'Karim Janat', 'Nangeyalia Kharote', 'Mujeeb Ur Rahman', 'Noor Ahmad', 'Naveen-ul-Haq', 'Fazalhaq Farooqi', 'Fareed Ahmad', nan, nan]


In [57]:
team_dict = {key.strip(): [value.strip() for value in values if pd.notna(value)] for key, values in team_dict.items()}

team_dict['Afghanistan']

['Rashid Khan',
 'Ibrahim Zadran',
 'Azmatullah Omarzai',
 'Najibullah Zadran',
 'Mohammad Ishaq',
 'Mohammad Nabi',
 'Gulbadin Naib',
 'Karim Janat',
 'Nangeyalia Kharote',
 'Mujeeb Ur Rahman',
 'Noor Ahmad',
 'Naveen-ul-Haq',
 'Fazalhaq Farooqi',
 'Fareed Ahmad']

In [8]:
players_list = np.concatenate((df['bowl'].unique(), df['bat'].unique()))
for team,players in team_dict.items():
    for player in players:
        if ((player not in players_list)|(player not in players_list)):
            print(team,player)

Canada Kanwarpal Tathgur
Namibia Jack Brassell
Namibia Malan Kruger
Namibia Peter-Daniel Blignaut
Nepal Anil Kumar Sah
Netherlands Daniel Doram
Netherlands Michael Levitt
Oman Khalid Kail
Scotland Jack Jarvis
Scotland Charlie Tear
United States Harmeet Singh
United States Nosthush Kenjige
United States Shayan Jahangir


In [58]:
players=set()
for key,vals in team_dict.items():
    players.update(vals)

In [59]:
matches_relevant=df[df['bat'].isin(players) | df['bowl'].isin(players)]['p_match'].unique()
df=df[df['p_match'].isin(matches_relevant)]

In [60]:
df_player_id_combs=df[['bat','p_bat','bowl','p_bowl']].drop_duplicates()

In [61]:
# Create a mapping of player names to player IDs
player_id_map = {}
for index, row in df_player_id_combs.iterrows():
    if row['bat'] in player_id_map:
        continue
    player_id_map[row['bat']] = row['p_bat']
    player_id_map[row['bowl']] = row['p_bowl']
# Initialize lists to store data
player_names = []
countries = []
player_ids = []

# Iterate over the dictionary
for country, players in team_dict.items():
    for player in players:
        player_names.append(player)
        countries.append(country)
        player_ids.append(player_id_map.get(player, None))

# Create DataFrame from lists
team_df = pd.DataFrame({'Player': player_names, 'Country': countries, 'p_id': player_ids})
team_df['p_id'] = team_df['p_id'].astype('Int64')
team_df.head()

Player      Country     p_id
0         Rashid Khan  Afghanistan   793463
1      Ibrahim Zadran  Afghanistan   921509
2  Azmatullah Omarzai  Afghanistan   819429
3   Najibullah Zadran  Afghanistan   524049
4      Mohammad Ishaq  Afghanistan  1076579

In [62]:
worldcup_pitches=['Kensington Oval, Bridgetown, Barbados',
                  'Sir Vivian Richards Stadium, North Sound, Antigua',
                  'Providence Stadium, Guyana',
                  'Daren Sammy National Cricket Stadium, Gros Islet, St Lucia',
                  'Brian Lara Stadium, Tarouba, Trinidad',
                  'Arnos Vale Ground, Kingstown, St Vincent',
                  'Central Broward Regional Park Stadium Turf Ground, Lauderhill',
                  'Nassau County International Cricket Stadium',
                  'Grand Prairie Stadium'
                 ]

In [63]:
df.columns

Index(['p_match', 'inns', 'bat', 'p_bat', 'team_bat', 'bowl', 'p_bowl',
       'team_bowl', 'ball', 'ball_id', 'outcome', 'score', 'out', 'dismissal',
       'p_out', 'over', 'noball', 'wide', 'byes', 'legbyes', 'cur_bat_runs',
       'cur_bat_bf', 'cur_bowl_ovr', 'cur_bowl_wkts', 'cur_bowl_runs',
       'inns_runs', 'inns_wkts', 'inns_balls', 'inns_runs_rem',
       'inns_balls_rem', 'inns_rr', 'inns_rrr', 'target', 'max_balls', 'date',
       'year', 'ground', 'country', 'winner', 'toss', 'competition',
       'bat_hand', 'bowl_style', 'bowl_kind', 'batruns', 'ballfaced',
       'bowlruns', 'bat_out', 'wagonX', 'wagonY', 'wagonZone', 'pitchLine',
       'pitchLength', 'shotType'],
      dtype='object')

In [64]:

def custom_agg(x):
    result = (x['batruns'].sum() * 120) / x['ballfaced'].sum()
    return round(result, 2)
def get_ground_avgscores(df):
    df1=df[df['ground'].isin(worldcup_pitches)].groupby(['p_match', 'inns','ground']).agg({'batruns':'sum','ballfaced':'sum'}).reset_index()
    # Define custom aggregation function
    # Group by 'inns' and 'ground', and aggregate using custom aggregation function
    result_df = df1.groupby(['inns', 'ground']).apply(custom_agg).reset_index(name='average_score')

    return result_df
df_groundavg=get_ground_avgscores(df)
df_groundavg.head()

inns                                             ground  average_score
0     1              Brian Lara Stadium, Tarouba, Trinidad         140.89
1     1  Central Broward Regional Park Stadium Turf Gro...         159.59
2     1  Daren Sammy National Cricket Stadium, Gros Isl...         157.40
3     1              Kensington Oval, Bridgetown, Barbados         150.04
4     1                         Providence Stadium, Guyana         145.95

In [65]:
def get_ground_batstats_vs_bowlkind(df):
    df2=df[(df['ground'].isin(worldcup_pitches))&(df['bowl_kind'].isin(['pace bowler','spin bowler']))]
    df2_runs=df2.groupby(['ground','bowl_kind']).agg({'batruns':'sum','ballfaced':'sum','p_match':'nunique'}).reset_index()
    df2_outs=df[(df['out']==True)&(df['ground'].isin(worldcup_pitches))&(df['bowl_kind'].isin(['pace bowler','spin bowler']))].groupby(['ground','bowl_kind']).size().reset_index(name='outs')
    df2_batting=pd.merge(df2_runs,df2_outs,on=['ground','bowl_kind'])
    df2_batting['average']=round(df2_batting['batruns']/df2_batting['outs'],2)
    df2_batting['strike rate']=round(df2_batting['batruns']*100/df2_batting['ballfaced'],2)
    df2_batting.drop(columns=['batruns','ballfaced','outs'],inplace=True)
    df2_batting.rename(columns={'p_match':'matches'},inplace=True)
    return df2_batting
df2_batting=get_ground_batstats_vs_bowlkind(df)
df2_batting.head()

ground    bowl_kind  matches  \
0              Brian Lara Stadium, Tarouba, Trinidad  pace bowler       43   
1              Brian Lara Stadium, Tarouba, Trinidad  spin bowler       43   
2  Central Broward Regional Park Stadium Turf Gro...  pace bowler       21   
3  Central Broward Regional Park Stadium Turf Gro...  spin bowler       21   
4  Daren Sammy National Cricket Stadium, Gros Isl...  pace bowler       15   

   average  strike rate  
0    20.98       128.26  
1    20.71       106.38  
2    25.00       138.15  
3    23.08       113.52  
4    20.67       131.66

In [66]:
def get_battingstats(df):
    df3=pd.merge(df[df['bat'].isin(team_df[team_df['p_id'].isna()==False]['Player'])],team_df[team_df['p_id'].isna()==False],left_on='p_bat',right_on='p_id',how='inner')
    df3_bat=df3.groupby(['Country']).agg({'batruns':'sum','ballfaced':'sum','p_match':'nunique'}).reset_index()
    df3_outs=df3[df3['out']==True].groupby(['Country']).size().reset_index(name='outs')
    df3_bat_stats=pd.merge(df3_bat,df3_outs,on='Country')
    df3_bat_stats['bat_avg']=round(df3_bat_stats['batruns']/df3_bat_stats['outs'],2)
    df3_bat_stats['bat_sr']=round(df3_bat_stats['batruns']*100/df3_bat_stats['ballfaced'],2)
    df3_dots=df3[(df3['bowlruns']==0) & (df3['ballfaced']==1)].groupby(['Country']).agg({'ballfaced':'sum'}).reset_index()
    df3_dots.rename(columns={'ballfaced':'dotballs'},inplace=True)
    df3_bat_stats=pd.merge(df3_bat_stats,df3_dots,on="Country")
    df3_bat_stats['dot%']=(df3_bat_stats['dotballs']*100/df3_bat_stats['ballfaced']).round(2)
    def custom_agg(x):
        boundary_runs = x['batruns'].sum()
        return boundary_runs
    # Apply the custom aggregation function and reset index
    df3_bd = df3[((df3['batruns'] == 4) | (df3['batruns'] == 6)) & (df3['ballfaced'] == 1)] \
                .groupby(['Country']).apply(custom_agg).reset_index(name='boundary_runs')
    df3_bat_stats=pd.merge(df3_bat_stats,df3_bd,on="Country")
    df3_bat_stats['boundary%']=(df3_bat_stats['boundary_runs']*100/df3_bat_stats['batruns']).round(2)
    df3_bat_stats.drop(columns=['batruns', 'ballfaced', 'dotballs','boundary_runs','outs'],inplace=True)
    df3_bat_stats.rename(columns={'p_match':'matches_batted'},inplace=True)
    return df3_bat_stats
df3_bat_stats=get_battingstats(df)
df3_bat_stats.head()

Country  matches_batted  bat_avg  bat_sr   dot%  boundary%
0  Afghanistan             620    20.39  134.57  37.87      60.17
1    Australia             949    29.37  142.22  33.67      59.40
2   Bangladesh             571    21.80  120.53  38.44      55.48
3       Canada              42    31.29  137.71  37.11      59.86
4      England            1144    27.48  144.67  34.51      62.91

In [67]:
def get_bowling_stats(df):
    df4=pd.merge(df[df['bowl'].isin(team_df[team_df['p_id'].isna()==False]['Player'])],team_df[team_df['p_id'].isna()==False],left_on='p_bowl',right_on='p_id',how='inner')
    df4_bowl=df4.groupby(['Country']).agg({'bowlruns':'sum','ballfaced':'sum','p_match':'nunique'}).reset_index()
    df4_bowl['economy']=(df4_bowl['bowlruns']*6/df4_bowl['ballfaced']).round(2)
    df4_outs=df4[df4['out']==True].groupby(['Country']).size().reset_index(name='outs')
    df4_bowl_stats=pd.merge(df4_bowl,df4_outs,on='Country')
    df4_bowl_stats['bowl_avg']=round(df4_bowl_stats['bowlruns']/df4_bowl_stats['outs'],2)
    df4_bowl_stats['bowl_sr']=round(df4_bowl_stats['ballfaced']/df4_bowl_stats['outs'],2)
    df4_bowl_stats.drop(columns=['bowlruns','ballfaced','outs'],inplace=True)
    df4_bowl_stats.rename(columns={'p_match':'matches_bowled'},inplace=True)
    return df4_bowl_stats
df4_bowl_stats=get_bowling_stats(df)
df4_bowl_stats.head()

Country  matches_bowled  economy  bowl_avg  bowl_sr
0  Afghanistan             897     7.08     21.10    17.88
1    Australia             745     7.86     24.03    18.35
2   Bangladesh             605     7.37     20.98    17.07
3       Canada              41     6.56     17.82    16.29
4      England             870     8.07     22.80    16.95

In [70]:
overall_team_stats=pd.merge(df3_bat_stats,df4_bowl_stats,on='Country').sort_values('matches_batted',ascending=False).reset_index().drop(columns='index').head(10)
overall_team_stats.drop(columns=['matches_batted','matches_bowled'],inplace=True)
overall_team_stats.set_index('Country', inplace=True)
overall_team_stats

bat_avg  bat_sr   dot%  boundary%  economy  bowl_avg  bowl_sr
Country                                                                    
England         27.48  144.67  34.51      62.91     8.07     22.80    16.95
Australia       29.37  142.22  33.67      59.40     7.86     24.03    18.35
West Indies     25.43  139.27  39.74      66.03     8.23     23.07    16.82
New Zealand     28.13  136.46  34.03      57.67     8.06     22.85    17.00
South Africa    28.31  134.91  34.50      58.12     7.42     21.08    17.04
India           30.58  139.00  34.18      60.36     7.77     23.91    18.46
Afghanistan     20.39  134.57  37.87      60.17     7.08     21.10    17.88
Bangladesh      21.80  120.53  38.44      55.48     7.37     20.98    17.07
Sri Lanka       25.56  127.08  36.49      56.39     7.55     23.57    18.72
Ireland         21.82  133.39  37.64      60.86     7.82     21.53    16.53

In [97]:
cols_increase=['bat_avg','bat_sr','boundary%']
cols_decrease=['dot%','economy','bowl_avg','bowl_sr']
for c in cols_increase:
    normalized_df[c] = (overall_team_stats[c] - overall_team_stats[c].mean()) / overall_team_stats[c].std()

for c in cols_decrease:
    normalized_df[c] = (overall_team_stats[c].mean() - overall_team_stats[c]) / overall_team_stats[c].std()

normalized_df

bat_avg    bat_sr      dot%  boundary%   economy  bowl_avg  \
Country                                                                     
West Indies  -0.129912  0.570098 -1.655442   2.013795 -1.399477 -0.476576   
India         1.334086  0.532185  0.878073   0.198787 -0.129735 -1.169177   
Australia     0.990117  0.984331  1.110463  -0.108516 -0.378162 -1.268120   
England       0.452844  1.328355  0.727702   1.015060 -0.957827 -0.253954   
Sri Lanka    -0.092957 -1.141599 -0.174521  -1.072039  0.477533 -0.888839   
New Zealand   0.637621  0.175523  0.946423  -0.662302 -0.930224 -0.295180   
Ireland      -1.156132 -0.255561 -0.698539   0.358840 -0.267750  0.793194   
South Africa  0.688790 -0.042125  0.732259  -0.518254  0.836374  1.164230   
Afghanistan  -1.562640 -0.089868 -0.803343   0.137966  1.774879  1.147740   
Bangladesh   -1.161817 -2.061338 -1.063074  -1.363337  0.974389  1.246683   

               bowl_sr  
Country                 
West Indies   0.837300  
India        -1.236977  
Australia    -1.097849  
England       0.672875  
Sri Lanka    -1.565826  
New Zealand   0.609635  
Ireland       1.204093  
South Africa  0.559043  
Afghanistan  -0.503392  
Bangladesh    0.521099

In [99]:
# Rank the data
ranked_df = normalized_df.rank(axis=0, method='min', ascending=False).astype(int)
ranked_df['overallrank'] = ranked_df.mean(axis=1)

# Rank the average ranks
ranked_df['overallrank'] = ranked_df['overallrank'].rank(method='min', ascending=True).astype(int)

# Sort the DataFrame based on the overall rank
ranked_df = ranked_df.sort_values(by='overallrank')
normalized_df=normalized_df.reindex(ranked_df.index)

plt.figure(figsize=(12, 8))
sns.heatmap(normalized_df, annot=ranked_df.drop(columns='overallrank'), fmt='d', cmap='RdYlGn', linewidths=.5, center=0)

# Title and labels
plt.title('Overall Team Stats Heatmap (Ranked)')
plt.xlabel('Statistics')
plt.ylabel('Country')

# Show the plot
plt.show()

In [100]:
# Calculate average rank
ranked_df['overallrank'] = ranked_df.mean(axis=1)

# Rank the average ranks
ranked_df['overallrank'] = ranked_df['overallrank'].rank(method='min', ascending=True).astype(int)

# Sort the DataFrame based on the overall rank
ranked_df = ranked_df.sort_values(by='overallrank')
ranked_df.index

Index(['England', 'South Africa', 'India', 'Australia', 'New Zealand',
       'Ireland', 'West Indies', 'Afghanistan', 'Bangladesh', 'Sri Lanka'],
      dtype='object', name='Country')

In [19]:
get_battingstats(df).head()

Country  matches  average  strike rate  dotballs%  boundary%
0  Afghanistan      620    20.39       134.57      37.87      60.17
1    Australia      949    29.37       142.22      33.67      59.40
2   Bangladesh      571    21.80       120.53      38.44      55.48
3       Canada       42    31.29       137.71      37.11      59.86
4     England      1144    27.48       144.67      34.51      62.91

In [33]:
get_battingstats(df[df['over']<=5]).head()

Country  matches  average  strike rate  dotballs%  boundary%
0  Afghanistan      100    27.00       122.59      51.78      72.31
1    Australia      513    35.91       132.15      44.31      67.47
2   Bangladesh      405    24.37       114.02      48.70      65.06
3       Canada       35    40.67       122.20      49.42      66.94
4     England       765    33.19       142.65      43.96      72.54

In [34]:
get_battingstats(df[(df['over']>=6)&(df['over']<=14)]).head()

Country  matches  average  strike rate  dotballs%  boundary%
0  Afghanistan      338    25.57       112.18      40.11      50.95
1    Australia      716    32.44       134.89      31.63      54.00
2   Bangladesh      464    24.89       116.72      34.82      48.92
3       Canada       39    36.07       123.33      37.16      52.50
4     England       804    31.15       137.60      30.80      56.16

In [36]:
get_battingstats(df[(df['over']>=15)])

Country  matches  average  strike rate  dotballs%  boundary%
0        Afghanistan      520    17.59       156.21      33.62      64.42
1          Australia      589    23.21       164.05      28.30      61.93
2         Bangladesh      380    16.13       137.61      33.02      57.48
3             Canada       35    24.31       174.54      27.42      64.92
4           England       631    19.28       162.98      29.58      63.78
5              India      538    24.91       163.58      28.35      63.57
6            Ireland      180    15.94       142.51      33.53      58.64
7            Namibia      219    21.64       150.30      31.78      59.79
8              Nepal       56    23.90       168.82      29.06      63.60
9        Netherlands      135    15.30       130.74      33.41      51.23
10       New Zealand      482    20.86       158.07      27.94      59.27
11              Oman       60    14.69       131.64      34.38      55.06
12  Papua New Guinea       45    17.55       143.85      33.56      54.87
13          Scotland       85    19.18       143.67      29.81      57.25
14      South Africa      456    22.25       158.51      28.15      60.99
15         Sri Lanka      258    18.61       151.17      32.69      63.07
16            Uganda       63    18.98       143.52      33.73      59.09
17     United States      104    19.25       154.49      31.57      58.78
18       West Indies      616    23.21       169.74      33.26      70.51